In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import openpyxl


from datetime import datetime
month_dict = {
    'Januar': '01',
    'Februar': '02',
    'März': '03',
    'April': '04',
    'Mai': '05',
    'Juni': '06',
    'Juli': '07',
    'August': '08',
    'September': '09',
    'Oktober': '10',
    'November': '11',
    'Dezember': '12'
}

In [ ]:
urls = [
    #{'url':'https://www.westmetall.com/de/markdaten.php?action=table&field=ACI', 'symbol':'MBI01', 'boerse': 'mbi'},
    #{'url':'https://www.westmetall.com/de/markdaten.php?action=table&field=Euro_FX1', 'symbol':'MBI02', 'boerse': 'mbi'},
    {'url':'https://www.westmetall.com/de/markdaten.php?action=table&field=WM_Cu_low', 'symbol':'MBI0543', 'boerse': 'mbi'},
    {'url':'https://www.westmetall.com/de/markdaten.php?action=table&field=WM_Cu_high', 'symbol':'MBI0544', 'boerse': 'mbi'}      
]

In [ ]:
all_data = []

for item in urls:
    url = item['url']
    symbol = item['symbol']
    boerse = item['boerse']

    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find("table")

        data = []
        headers = ["symbol", "boerse", "datum", "letzter"]
    
        rows = table.find_all("tr")

        for row in table.find_all("tr")[1:4]:
            row_data = [cell.get_text(strip=True) for cell in row.find_all("td")]
            
            if len(row_data) >= 2:
                date_parts = row_data[0].split(' ')
                day = date_parts[0].strip('.')
                month = month_dict.get(date_parts[1])
                year = date_parts[2]

                datum = f"{year}-{month}-{day}"
                data.append([symbol, boerse, datum, row_data[1]])
        
        
        all_data.extend(data)

df = pd.DataFrame(all_data, columns=headers)
df.to_excel("westmetall.xlsx", index=False)

print("Excel-Datei erfolgreich erstellt: westmetall.xlsx")